1. Connect GPU (standard)
2. Connect high-RAM runtime

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Nov 25 00:02:57 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install flask
!pip install flask-ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install pyngrok
!ngrok authtoken '2FRxy0n5zXiQ25PPAkJXz2ksB8L_5cZPRefZhcKXqKpaNSw78'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 745 kB 44.6 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19006 sha256=25f647e30bfc608403c8da8c1a7b66ce139919a4714ac33257f5b44a0205bf92
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


# OD Environment

In [ ]:
!git clone https://github.com/pjreddie/darknet.git
!git clone https://github.com/bennycheung/Food100_YOLO_Tools.git

Cloning into 'darknet'...
remote: Enumerating objects: 5955, done.
remote: Total 5955 (delta 0), reused 0 (delta 0), pack-reused 5955
Receiving objects: 100% (5955/5955), 6.37 MiB | 6.53 MiB/s, done.
Resolving deltas: 100% (3932/3932), done.
Cloning into 'Food100_YOLO_Tools'...
remote: Enumerating objects: 15, done.
remote: Total 15 (delta 0), reused 0 (delta 0), pack-reused 15
Unpacking objects: 100% (15/15), done.


**prerequisites**
* `darknet/Makefile`
  * LIBSO=1
* `darknet/python/darknet.py` 
  * lib = CDLL("/content/darknet/libdarknet.so", RTLD_GLOBAL)


In [ ]:
!cp -r /content/drive/MyDrive/AIDiet_data/ /content
!cp /content/AIDiet_data/Makefile /content/darknet/Makefile
!cp /content/AIDiet_data/darknet.py /content/darknet/python/darknet.py
!cp /content/AIDiet_data/food100.data /content/Food100_YOLO_Tools/food100.data

In [ ]:
%cd /content/darknet/
!make

/content/darknet
gcc -Iinclude/ -Isrc/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -c ./src/gemm.c -o obj/gemm.o
gcc -Iinclude/ -Isrc/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -c ./src/utils.c -o obj/utils.o
gcc -Iinclude/ -Isrc/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -c ./src/cuda.c -o obj/cuda.o
gcc -Iinclude/ -Isrc/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -c ./src/deconvolutional_layer.c -o obj/deconvolutional_layer.o
gcc -Iinclude/ -Isrc/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -c ./src/convolutional_layer.c -o obj/convolutional_layer.o
gcc -Iinclude/ -Isrc/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -c ./src/list.c -o obj/list.o
gcc -Iinclude/ -Isrc/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -c ./src/image.c -o obj/image.o
gcc -Iinclude/ -Isrc/ -Wall -Wno-

# Test

In [ ]:
import sys, os
sys.path.insert(1, '/content/darknet/python/')
import darknet as dn
import pdb

dn.set_gpu(0)
net = dn.load_net(b"/content/Food100_YOLO_Tools/yolov2-food100.cfg", b"/content/AIDiet_data/yolov2-food100.weights", 0)
meta = dn.load_meta(b"/content/Food100_YOLO_Tools/food100.data")
file_name = "/content/AIDiet_data/hamburger.jpg"
r = dn.detect(net, meta, file_name.encode())
print(r)

response = {}
response['food_name'] = r[0][0].decode('utf-8')
response['confidence'] = r[0][1]
response['bb'] = r[0][2]

[(b'hamburger', 0.9913035035133362, (252.65716552734375, 213.94979858398438, 480.865966796875, 537.388671875))]


In [ ]:
import cv2

im = cv2.imread(file_name)
print(im.shape)
width, height = im.shape[0], im.shape[1]

focal_length = 26
sensor_height=0.0014
distance = 200 #m -> cm

w, h = 252-213, 537-480
print(w, h)

width = w
height = h
width * sensor_height * distance / focal_length , height * sensor_height * distance / focal_length

(444, 512, 3)
39 57


(0.42, 0.6138461538461538)

# Object Detection

Result Image !!!!:)

In [ ]:
import sys, os
sys.path.insert(1, '/content/darknet/python/')
import darknet as dn
import pdb

from flask import *
from flask_ngrok import run_with_ngrok
app = Flask(__name__)

import time
  

@app.route('/detect')
def detectFood():
  start_time = time.time()
  dn.set_gpu(0)
  net = dn.load_net(b"/content/Food100_YOLO_Tools/yolov2-food100.cfg", b"/content/AIDiet_data/yolov2-food100.weights", 0)
  meta = dn.load_meta(b"/content/Food100_YOLO_Tools/food100.data")
  file_name = "/content/AIDiet_data/hamburger.jpg"
  r = dn.detect(net, meta, file_name.encode())
  print(r)

  response = {}
  response['food_name'] = r[0][0].decode('utf-8')
  response['confidence'] = r[0][1]
  response['bb'] = r[0][2]
  response['elapsed_time'] = time.time()-start_time

  return response

# Segmentation

In [ ]:
%cd /content/
!git clone https://github.com/LARC-CMU-SMU/FoodSeg103-Benchmark-v1.git
!pip install -q condacolab

/content
fatal: destination path 'FoodSeg103-Benchmark-v1' already exists and is not an empty directory.


Exception in thread _colab_inspector_thread:
Traceback (most recent call last):
  File "/usr/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.7/dist-packages/google/colab/_debugpy.py", line 64, in inspector_thread
    _variable_inspector.run(shell, time)
  File "/usr/local/lib/python3.7/dist-packages/google/colab/_variable_inspector.py", line 27, in run
    globals().clear()
TypeError: 'module' object is not callable



In [ ]:
!rm -rf /content/FoodSeg103-Benchmark-v1/checkpoints
!mkdir /content/FoodSeg103-Benchmark-v1/checkpoints/
!cp /content/AIDiet_data/pspnet_r50-d8_512x1024_40k_cityscapes_20200605_003338-2966598c.pth /content/FoodSeg103-Benchmark-v1/checkpoints/

In [ ]:
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [ ]:
import condacolab
condacolab.check()

✨🍰✨ Everything looks OK!


In [ ]:
!mamba install -q openmm
!conda create -n open-mmlab python=3.7 -y
!conda activate open-mmlab

!conda install pytorch=1.6.0 torchvision cudatoolkit=10.1 -c pytorch

!pip uninstall mmcv-full
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu101/torch1.6.0/index.html

!pip install mmsegmentation # install the latest release

!pip install -r /content/FoodSeg103-Benchmark-v1/requirements.txt


                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔██║███████║██╔████╔██║██████╔╝███████║
        ██║╚██╔╝██║██╔══██║██║╚██╔╝██║██╔══██╗██╔══██║
        ██║ ╚═╝ ██║██║  ██║██║ ╚═╝ ██║██████╔╝██║  ██║
        ╚═╝     ╚═╝╚═╝  ╚═╝╚═╝     ╚═╝╚═════╝ ╚═╝  ╚═╝

        mamba (0.25.0) supported by @QuantStack

        GitHub:  https://github.com/mamba-org/mamba
        Twitter: https://twitter.com/QuantStack

█████████████████████████████████████████████████████████████

  Package              Version  Build                Channel                   Size
─────────────────────────────

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 828.7/828.7 kB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 105.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 648.0/648.0 kB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.7/247.7 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 67.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 965.4/965.4 kB 62.4 MB/s eta 0:00:00


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.3/473.3 kB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.8/316.8 kB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.1/210.1 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.1/66.1

In [ ]:
import pandas as pd
df = pd.read_csv('/content/AIDiet_data/category_id.txt', sep = ',', names=[ 'name', 'shape'], index_col=0)

# Segmentation REsult

In [ ]:
from mmseg.apis import init_segmentor, inference_segmentor, show_result_pyplot
from mmseg.core.evaluation import get_palette

import numpy as np
def show_result(model, img, result):
  import mmcv
  from google.colab.patches import cv2_imshow
  
  palette=None
  win_name=''
  show=False
  wait_time=0
  out_file=None

  img = mmcv.imread(img)
  img = img.copy()
  seg = result[0]
  if palette is None:
      if model.PALETTE is None:
          palette = np.random.randint(
              0, 255, size=(len(model.CLASSES), 3))
      else:
          palette = model.PALETTE
  palette = np.array(palette)
  assert palette.shape[0] == len(model.CLASSES)
  assert palette.shape[1] == 3
  assert len(palette.shape) == 2
  color_seg = np.zeros((seg.shape[0], seg.shape[1], 3), dtype=np.uint8)
  results = []
  for label, color in enumerate(palette):
      # Check seg labels!!
      if(len(color_seg[seg == label, :]) > 0):
        name = df.iloc[label]['name']
        if name in ('background','other ingredients'): 
          continue
        portion = len(color_seg[seg == label, :])
        results.append([name, portion, color])
      color_seg[seg == label, :] = color
  # convert to BGR
  color_seg = color_seg[..., ::-1]

  img = img * 0.5 + color_seg * 0.5
  img = img.astype(np.uint8)
  # if out_file specified, do not show image in window
  if out_file is not None:
      show = False

  if show:
      cv2_imshow(img, win_name, wait_time)
  if out_file is not None: ##Image Save-----------------------------------------------------------------
      mmcv.imwrite(img, out_file)

  if not (show or out_file):
      print('show==False and out_file is not specified, only '
                    'result image will be returned')
  return results
      
def softmax(x):
  return x /sum(x)

import json
import requests

def get_caloies(name, volume):
  query = f'{volume}g {name}'
  api_url = f'https://api.api-ninjas.com/v1/nutrition?query={query}'
  print(query)
  response = requests.get(api_url, headers={'X-Api-Key': 'uF82Rnp+3IHaDuVRJ157GA==We7LxVlYDzefzxtx'})
  if response.status_code == requests.codes.ok:
      api = json.loads(response.content)
      if len(api) != 0:
        #calories= api[0]['calories']
        return api
  else:
      print("Error:", response.status_code, response.text)
  ##if ingredient doesn't exist in API
  print(f'not passed {query}')
  return 0


In [ ]:
import sys, os
sys.path.insert(1, '/content/darknet/python/')
import darknet as dn
import pdb
import json

from flask import Flask, request
from flask_ngrok import run_with_ngrok
app = Flask(__name__)

import time
import json


@app.route('/calories', methods=['POST'])
def calories():
  image = request.files['image']
  image.save(image.filename)
  start_time = time.time()

  config_file = '/content/AIDiet_data/checkpoints/CCNet_ReLeM/ccnet_r50-d8_512x1024_80k.py'
  checkpoint_file = '/content/AIDiet_data/checkpoints/CCNet_ReLeM/iter_80000.pth'
  model = init_segmentor(config_file, checkpoint_file, device='cuda:0')

  #file_name = '/content/AIDiet_data/hamburger.jpg'
  file_name = f'/content/{image.filename}'
  result = inference_segmentor(model, file_name)
  ing_results = show_result(model, file_name, result)
  ing_results.sort(key=lambda x:x[1], reverse=True)

  # Normalization
  ratios = np.array([int(f[1]) for f in ing_results])
  ratios = softmax(ratios)

  # Calculate and Summation
  calories_list = []
  carbohydrates_list = []
  cholesterol_list = []
  fat_saturated_list = []
  fat_total_list = []
  fiber_list = []
  potassium_list = []
  protein_list = []
  sodium_list = []
  sugar_list = []
  volume_list = []
  VOLUME = 250 #-------------------------------------------------list에서 가져올까?
  for i, food in enumerate(ing_results):
    volume = int(ratios[i]*VOLUME)
    volume_list.append(volume)
    result = get_caloies(name=food[0], volume=volume)
    if isinstance(result, int): 
      print(f'CHECK : {food}')
      continue
    api = result[0]
    calories_list.append(api['calories'])
    carbohydrates_list.append(api['carbohydrates_total_g'])
    cholesterol_list.append(api['cholesterol_mg'])
    fat_saturated_list.append(api['fat_saturated_g'])
    fat_total_list.append(api['fat_total_g'])
    fiber_list.append(api['fiber_g'])
    potassium_list.append(api['potassium_mg'])
    protein_list.append(api['protein_g'])
    sodium_list.append(api['sodium_mg'])
    sugar_list.append(api['sugar_g'])

  response = {}
  items = []
  #response['result'] = result
  for i in range(len(ing_results)):
    item = {}
    item['ingredient'] = ing_results[i][0]
    item['ratio'] = round(ratios[i], 2)
    item['volume'] = volume_list[i]
    item['calories'] = calories_list[i]
    items.append(item)
  response['items'] = items
  response['serving_size_g'] = int(VOLUME)
  response['calories'] = int(sum(calories_list))
  response['carbohydrates_total_g'] = int(sum(carbohydrates_list))
  response['cholesterol_mg'] = int(sum(cholesterol_list))
  response['fat_saturated_g'] = int(sum(fat_saturated_list))
  response['fat_total_g'] = int(sum(fat_total_list))
  response['fiber_g'] = int(sum(fiber_list))
  response['potassium_mg'] = int(sum(potassium_list))
  response['protein_g'] = int(sum(protein_list))
  response['sodium_mg'] = int(sum(sodium_list))
  response['sugar_g'] = int(sum(sugar_list))

  print(response)
  return json.dumps(response)

# API

In [ ]:
run_with_ngrok(app)
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://a23a-34-80-148-202.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [11/Nov/2022 20:00:27] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [11/Nov/2022 20:00:27] "GET /favicon.ico HTTP/1.1" 404 -
/usr/local/lib/python3.7/site-packages/mmseg/models/losses/cross_entropy_loss.py:236: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  'Default ``avg_non_ignore`` is False, if you would like to '


load checkpoint from local path: /content/AIDiet_data/checkpoints/CCNet_ReLeM/iter_80000.pth
show==False and out_file is not specified, only result image will be returned
88g lettuce
79g bread
28g steak
20g tomato
18g onion
6g cheese butter
3g sauce
3g pepper


...

{'items': [{'ingredient': 'bread', 'ratio': 0.37, 'volume': 92, 'calories': 240.7}, {'ingredient': 'lettuce', 'ratio': 0.21, 'volume': 52, 'calories': 8.9}, {'ingredient': 'steak', 'ratio': 0.19, 'volume': 47, 'calories': 128.5}, {'ingredient': 'cheese butter', 'ratio': 0.08, 'volume': 19, 'calories': 74.8}, {'ingredient': 'egg', 'ratio': 0.05, 'volume': 13, 'calories': 19.1}, {'ingredient': 'chicken duck', 'ratio': 0.04, 'volume': 10, 'calories': 22.3}, {'ingredient': 'pork', 'ratio': 0.03, 'volume': 7, 'calories': 16.5}, {'ingredient': 'sauce', 'ratio': 0.01, 'volume': 2, 'calories': 1.0}, {'ingredient': 'sausage', 'ratio': 0.01, 'volume': 2, 'calories': 6.4}, {'ingredient': 'cabbage', 'ratio': 0.0, 'volume': 0, 'calories': 0.2}, {'ingredient': 'pumpkin', 'ratio': 0.0, 'volume': 0, 'calories': 0.2}], 'serving_size_g': 250, 'calories': 518, 'carbohydrates_total_g': 48, 'cholesterol_mg': 127, 'fat_saturated_g': 8, 'fat_total_g': 22, 'fiber_g': 3, 'potassium_mg': 342, 'protein_g': 31, '

127.0.0.1 - - [11/Nov/2022 20:22:41] "POST /calories HTTP/1.1" 200 -


{'items': [{'ingredient': 'lettuce', 'ratio': 0.38, 'volume': 94, 'calories': 16.0}, {'ingredient': 'bread', 'ratio': 0.36, 'volume': 88, 'calories': 230.2}, {'ingredient': 'lemon', 'ratio': 0.12, 'volume': 30, 'calories': 8.7}, {'ingredient': 'steak', 'ratio': 0.11, 'volume': 27, 'calories': 73.8}, {'ingredient': 'sausage', 'ratio': 0.03, 'volume': 8, 'calories': 25.8}, {'ingredient': 'chicken duck', 'ratio': 0.0, 'volume': 0, 'calories': 2.2}, {'ingredient': 'pork', 'ratio': 0.0, 'volume': 0, 'calories': 2.4}], 'serving_size_g': 250, 'calories': 359, 'carbohydrates_total_g': 50, 'cholesterol_mg': 31, 'fat_saturated_g': 3, 'fat_total_g': 10, 'fiber_g': 5, 'potassium_mg': 184, 'protein_g': 18, 'sodium_mg': 522, 'sugar_g': 6}
load checkpoint from local path: /content/AIDiet_data/checkpoints/CCNet_ReLeM/iter_80000.pth
show==False and out_file is not specified, only result image will be returned
129g bread
56g fried meat
25g lettuce
21g cheese butter
11g ice cream
2g chicken duck
1g fish


127.0.0.1 - - [11/Nov/2022 20:23:31] "POST /calories HTTP/1.1" 200 -


{'items': [{'ingredient': 'bread', 'ratio': 0.52, 'volume': 129, 'calories': 337.5}, {'ingredient': 'fried meat', 'ratio': 0.23, 'volume': 56, 'calories': 123.9}, {'ingredient': 'lettuce', 'ratio': 0.1, 'volume': 25, 'calories': 4.3}, {'ingredient': 'cheese butter', 'ratio': 0.09, 'volume': 21, 'calories': 82.7}, {'ingredient': 'ice cream', 'ratio': 0.04, 'volume': 11, 'calories': 22.8}, {'ingredient': 'chicken duck', 'ratio': 0.01, 'volume': 2, 'calories': 4.5}, {'ingredient': 'fish', 'ratio': 0.01, 'volume': 1, 'calories': 1.3}, {'ingredient': 'cilantro mint', 'ratio': 0.0, 'volume': 0, 'calories': 0.2}, {'ingredient': 'pie', 'ratio': 0.0, 'volume': 0, 'calories': 2.3}, {'ingredient': 'onion', 'ratio': 0.0, 'volume': 0, 'calories': 0.4}, {'ingredient': 'juice', 'ratio': 0.0, 'volume': 0, 'calories': 0.5}, {'ingredient': 'sausage', 'ratio': 0.0, 'volume': 0, 'calories': 3.2}], 'serving_size_g': 250, 'calories': 583, 'carbohydrates_total_g': 69, 'cholesterol_mg': 76, 'fat_saturated_g':

# Image Send 결과 사진 전달

In [ ]:
from flask import Flask, send_file
from flask_ngrok import run_with_ngrok
app = Flask(__name__)

@app.route('/resultImages')
def get_image():
    '''
    zipf = zipfile.ZipFile('Name.zip','w', zipfile.ZIP_DEFLATED)
    for root,dirs, files in os.walk('output/'):
        for file in files:
            zipf.write('output/'+file)
    zipf.close()
    return send_file('Name.zip',
            mimetype = 'zip',
            attachment_filename= 'Name.zip',
            as_attachment = True)'''
    filename='input.jpg'
    return send_file(filename, mimetype='image/jpg', as_attachment = True)

1.   사진 전송 구현 (API)
2. Calorie 추정 API 최종 구현
3.   focal length 부피측정

